# Minimal 1-Node Dispatch Example

The example models a simple electricity system as a 1-node-model the following nodes (buses and components):

![](dispatch.png?=20)


## Python Imports

Imports of necessary python packages

In [2]:
import os
import pkg_resources as pkg
import pandas as pd 

from oemof.solph import EnergySystem, Model, Bus
import oemof.tabular.tools.postprocessing as pp
import oemof.tabular.facades as fc

## Creating and Setting the Datapaths

Setting the datapath for raw-data and results. Data handling looks more complex than it is. You can easily adapt this to a simple `pd.read_excel(filepath,...)` in the next block if your file is located somewhere else. Otherwise we will use data from the oemof tabular repository. 

In addition a results directory will be created in `home/user/oemof-results/dispatch/output`. 

In [46]:
# datapath for input data from the oemof tabular pacakge
datapath = os.path.join(
    pkg.resource_filename("oemof.tabular", "").replace('src/oemof/tabular', ""), 
    "data/data.xls")

# results path for output
results_path = os.path.join(
    os.path.expanduser("~"), 
    "oemof-results", 
    "dispatch", 
    "output"
)

if not os.path.exists(results_path):
    os.makedirs(results_path)
datapath

'/home/admin/projects/oemof-tabular/data/data.xls'

Next we will read the required input data. The timeseries index will be used for the `EnergySystem` object below. 
All generator data etc. will also be loaded. 

In [65]:
timeseries = pd.read_excel(
    datapath, sheet_name="timeseries", index_col=[0], parse_dates=True
)
timeseries.index.freq = "1H"

generators = pd.read_excel(
    datapath, sheet_name="generators", index_col=0
)
storages = pd.read_excel(
    datapath, sheet_name="storages", index_col=0
)

TypeError: set expected at most 1 arguments, got 2

## Creating the EnergySystem and its Nodes

We are starting by creating a `EnergySystem` object which will hold all information (nodes, etc.) of hour energy system that we will add below. This is just the standard way of using the `oemof.solph` library for your modelling.

In [ ]:
es = EnergySystem(timeindex=timeseries.index)

### Add Bus

Before we add any components we will create all bus objects for our model and add it to the energy system object.

In [ ]:
myamar = Bus(label="MN")
es.add(myamar)

#### Bus Constraints 

With the set of all Buses $B$ all inputs $x^{flow}_{i(b),b}$ to a bus $b$ must equal all its outputs $x^{flow}_{b,o(b)}$

$$\sum_i x^{flow}_{i(b), b}(t) - \sum_o x^{flow}_{b, o(b)}(t) = 0 \qquad \forall t \in T, \forall b \in B$$

This equation will be build once the complete energy system is setup with its component. Every time a `Component` is created, the connected bus inputs/outputs will be updated. By this update every bus has all required information of its inputs and outputs available to construct the constraints. 


### Add Load


In [ ]:
es.add(
    fc.Load(label="load", bus=myamar, amount=500e3, profile=timeseries["load"]))


#### Load Constraint 

For the set of all Load denoted with $l \in L$ the load $x_l$ at timestep t equals the exogenously defined  profile value $c^{profile}_l$ multiplied by the amount of this load $c^{amount}_l$

$$ x^{flow}_{l}(t) = c^{profile}_{l}(t) \cdot c^{amount}_{l} \qquad \forall t \in T, \forall l \in L$$

### Add Generators

In [ ]:
es.add(
    fc.Generator(
        label="ccgt",
        bus=myamar,
        carrier="gas",
        tech="ccgt",
        capacity=100,
        marginal_cost=25,
    )
)

#### Dispatchable Generator Constraint

A `Generator` component can be used to model all types of dispatchble units in a energy system. This can include diesel generators oder coal fired power plants but also hot water boilers for heat. Every generator **must** be connected to an `Bus` object. 

This basic mathematical model for the component with the set of all dispatchable generators being $d \in D$ looks as follows:

$$x^{flow}_{d}(t) \leq c^{capacity}_{d} \qquad \forall t \in T,  \forall d \in D$$

Meaning, the production of the generator $x^{flow}$ must be less than its maximum capacity $c^{max}$ in every timestep. 

In [ ]:
es.add(
    fc.Generator(
        label="pv",
        carrier="solar",
        tech="pv",
        capacity=150,
        bus=myamar,
        output_parameters={
                'fixed': True},
        profile=timeseries["pv"],
    )
)

## same behaviour as above
# es.add(
#    fc.Volatile(
#        label="wind",
#        carrier="wind",
#        tech="onshore",
#        capacity=150,
#        bus=bus,
#        profile=timeseries["onshore"],
#    )
#)



#### Volatile Generator Constraint 

Using the `Generator` component with `output_parameters={"fixed": True}` is very similar to the Dispatchable component. However, in this case the flow of the volatile components denoted with $v \in V$  will be fixed to a specific value.

$$ x^{flow}_{v}(t) = c^{profile}_{v}(t) \cdot c^{capacity}_{v} \qquad \forall t \in T, \forall v \in V$$

Alternatively you can use the `Volatile` component which automatically enforced the fixed behaviour. 

### Add Storage

In [ ]:
es.add(
    fc.Storage(
        label="storage",
        bus=myamar,
        carrier="lithium",
        tech="battery",
        capacity=20,
        marginal_cost=0.0001,
        balanced=True,
        initial_storage_capacity=0.5,
        storage_capacity=100,
    )
)
es.add(fc.Excess(label="excess", bus=myamar))

#### Storage Constraints 

The mathematical representation of the storage for all storages $s \in S$ will include the flow into the storage, out of the storage and a storage level. The defaul efficiency for input/output is 1. Note that this is is included during charge and discharge. If you want to set the round trip efficiency you need to do for example: $\eta = \sqrt{\eta^{roundtrip}}$

Intertemporal energy balance of the storage:

$$ x^{level}_{s}(t) = \eta^{loss} x^{level}_{s}(t) + \eta x^{flow}_{s, in} - \eta x^{flow}_{s, out}(t) \qquad \forall t \in T,  \forall s \in S$$ 

Bounds of the storage level variable $x^{level}_s(t)$:

$$ x^{level}_s(t) \leq c_s^{max,level} \qquad \forall t \in T,  \forall s \in S$$


$$ x^{level}_s(1) = x_s^{level}(t_{e}) = 0.5 \cdot c_s^{max,level} \qquad \forall t \in T,  \forall s \in S$$ 

Of course, in addition the inflow/outflow of the storage also needs to be within the limit of the minimum and maximum power. 

$$ -c_s^{capacity} \leq x^{flow}_s(t) \leq c_s^{capacity} \qquad \forall t \in T, \forall s \in S$$ 


### Objective Function 

The objective function is created from all instantiated objects. It will use all operating costs (i.e. `marginal_cost` argument) and if set all investment costs (i.e. `capacity_cost` argument)

$$ \text{min:} \sum_g \sum_t \overbrace{c^{marginal\_cost}_g \cdot x^{flow}_{g}(t)}^{\text{operating cost}} \\ 
\sum_g \sum_t \overbrace{c^{capacity\_cost}_g \cdot x^{capacity}_{g}(t)}^{\text{investment cost}} $$

## Creating the Mathematical Model

In [ ]:
# create model based on energy system and its components
m = Model(es)

# inspect objective function
# m.objective.pprint()

## Solving the Model and Writing Results

In [ ]:

#  solve the model using cbc solver
m.solve("cbc")

# write results back to the model object
m.results = m.results()

# writing results with the standard oemof-tabular output formatt
pp.write_results(m, results_path)

print("Optimization done. Results are in {}.".format(results_path))


## Plotting the Results

In [ ]:
from plotly import offline, plotly
from oemof.tabular.tools.plots import hourly_plot
offline.init_notebook_mode()

name = "dispatch"

# results path for output
results_path = os.path.join(os.path.expanduser("~"), "oemof-results", name, "output")

offline.plot(
    hourly_plot(
        name,
        "MN",
        os.path.join(os.path.expanduser("~"), "oemof-results"),
        plot_filling_levels=False,
    ),
    filename=os.path.join(results_path, "hourly-plot.html"),
)